In [ ]:
import datasets
import pandas as pd


In [ ]:

dataset = datasets.load_dataset("SetFit/bbc-news")

train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])
combined_df = pd.concat([train_df, test_df])

print("Dataset Overview:")
print(f"Dataset size: {len(combined_df)}")
print("\nLabel distribution in training set:")
print(combined_df['label'].value_counts())
print("\nSample text from training set:")
print(combined_df['text'].iloc[0][:500])

categories = dataset['train'].features['label_text'].names
print("\nCategories:", categories)

train_df['text_length'] = train_df['text'].str.len()
print(f"\nText length mean: {train_df['text_length'].mean():.2f}")